In [9]:
import MeCab
import unidic
import pathlib

dic_path = pathlib.Path(unidic.DICDIR)
dic_dir = str(dic_path).replace("\\", "/")
mecab = MeCab.Tagger(f'-r "{dic_dir}/mecabrc" -d "{dic_dir}"')
keep_pos = {"名詞", "動詞", "形容詞", "副詞"}

def mecab_tokenizer(text: str):
    text = str(text).replace("\r", " ").replace("\n", " ")

    node = mecab.parseToNode(text)
    tokens = []

    while node:
        surface = node.surface
        # BOS/EOS や空文字はスキップ
        if not surface:
            node = node.next
            continue

        features = node.feature.split(",")
        pos = features[0]      # 品詞 大分類（名詞, 動詞, 助詞, など）

        # ★ 名詞・動詞・形容詞・副詞 以外は捨てる
        if pos not in keep_pos:
            node = node.next
            continue

        # ==== 原形（lemma）を優先して使う ====
        # UniDic 系の典型的な並び:
        #  0: pos1, 1: pos2, 2: pos3, 3: pos4,
        #  4: cType, 5: cForm, 6: lForm, 7: lemma, 8: orth, 9: pron, ...
        lemma_idx = 7
        orth_idx  = 8

        token = surface  # デフォルトは表層形

        if len(features) > lemma_idx and features[lemma_idx] not in ("*", ""):
            token = features[lemma_idx]   # 漢字混じりの原形
        elif len(features) > orth_idx and features[orth_idx] not in ("*", ""):
            token = features[orth_idx]    # 文中表記

        tokens.append(token)
        node = node.next

    return tokens

mecab_tokenizer("ああ、今日はなんていう日だ。お財布を会社に忘れてしまうなんて。。。")


['今日', '言う', '日', '財布', '会社', '忘れる', '仕舞う']

In [15]:
import numpy as np

############################################
# 2. ユーティリティ
############################################

def l2_normalize(x: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(x, axis=-1, keepdims=True) + 1e-12
    return x / norm

def log_softmax(x):
    x = np.asarray(x, dtype=np.float64)
    x_max = np.max(x)
    log_sum_exp = np.log(np.sum(np.exp(x - x_max))) + x_max
    return x - log_sum_exp


In [22]:
from sentence_transformers import SentenceTransformer

############################################
# 3. vMF 風のロバスト分類器
############################################

class RobustTextClassifier:
    """
    - 各カテゴリ k について、トークン埋め込みの平均方向 μ_k, 簡易集中度 κ_k を学習
      （vMF 分布の log p(x|k) ∝ κ_k * μ_k^T x を模している）
    - 社内語（inhouse_terms）は埋め込みには使わず、p(k|w) を別で推定して加点
    """
    dic_path = pathlib.Path(unidic.DICDIR)
    dic_dir = str(dic_path).replace("\\", "/")
    mecab = MeCab.Tagger(f'-r "{dic_dir}/mecabrc" -d "{dic_dir}"')

    def __init__(
        self,
        categories,
        st_model="stsb-xlm-r-multilingual",
        keep_pos: set[str] = {"名詞", "動詞", "形容詞", "副詞"},
        inhouse_terms = None,
        alpha_inhouse=1.0,
    ):
        """
        categories: カテゴリ名のリスト
        st_model: SentenceTransformer インスタンス
        tokenizer: str -> List[str] のトークン化関数
        inhouse_terms: 社内語（固有名詞など）の set
        alpha_inhouse: p(k|w) をどれくらい効かせるか
        """
        self.categories = list(categories)
        self.num_classes = len(categories)
        self.cat2idx = {c: i for i, c in enumerate(self.categories)}

        self.keep_pos = keep_pos
        self.st_model = SentenceTransformer(st_model)
        self.inhouse_terms = set(inhouse_terms or [])
        self.alpha_inhouse = alpha_inhouse

        # パラメータ（fit でセット）
        self.mu = None              # (K, D) クラスごとの平均方向
        self.kappa = None           # (K,) 簡易集中度
        self.class_priors = None    # (K,) p(k)
        self.inhouse_p_kw = {}      # dict[w] -> np.array shape (K,)


    def _tokenizer(self, text: str):
        text = str(text).replace("\r", " ").replace("\n", " ")

        node = self.mecab.parseToNode(text)
        tokens = []

        while node:
            surface = node.surface
            # BOS/EOS や空文字はスキップ
            if not surface:
                node = node.next
                continue

            features = node.feature.split(",")
            pos = features[0]      # 品詞 大分類（名詞, 動詞, 助詞, など）

            # ★ 名詞・動詞・形容詞・副詞 以外は捨てる
            if pos not in self.keep_pos:
                node = node.next
                continue

            # ==== 原形（lemma）を優先して使う ====
            # UniDic 系の典型的な並び:
            #  0: pos1, 1: pos2, 2: pos3, 3: pos4,
            #  4: cType, 5: cForm, 6: lForm, 7: lemma, 8: orth, 9: pron, ...
            lemma_idx = 7
            orth_idx  = 8

            token = surface  # デフォルトは表層形

            if len(features) > lemma_idx and features[lemma_idx] not in ("*", ""):
                token = features[lemma_idx]   # 漢字混じりの原形
            elif len(features) > orth_idx and features[orth_idx] not in ("*", ""):
                token = features[orth_idx]    # 文中表記

            tokens.append(token)
            node = node.next

        return tokens
    
    ########################################
    # 内部: トークン埋め込み
    ########################################
    def _embed_tokens(self, tokens):
        if not tokens:
            D = self.st_model.get_sentence_embedding_dimension()
            return np.zeros((0, D))
        vecs = self.st_model.encode(
            tokens,
            convert_to_numpy=True,
            normalize_embeddings=True,  # L2 正規化済み
        )
        return vecs

    ########################################
    # 学習
    ########################################
    def fit(self, docs, labels):
        """
        docs: 文書のリスト（文字列）
        labels: 各文書のカテゴリ名
        """
        label_idx = np.array([self.cat2idx[y] for y in labels])
        class_counts = np.bincount(label_idx, minlength=self.num_classes)
        self.class_priors = class_counts / class_counts.sum()

        D = self.st_model.get_sentence_embedding_dimension()
        class_vec_sums = np.zeros((self.num_classes, D), dtype=np.float64)
        class_vec_counts = np.zeros(self.num_classes, dtype=np.float64)

        # 社内語のカウント（p(k|w) 学習用）
        # inhouse_counts = defaultdict(lambda: np.zeros(self.num_classes, dtype=np.float64))

        for doc, y in zip(docs, labels):
            k = self.cat2idx[y]
            tokens = self._tokenizer(doc)

            normal_tokens = [t for t in tokens if t not in self.inhouse_terms]
            # inhouse_tokens = [t for t in tokens if t in self.inhouse_terms]

            # 社内語の出現カウント
            # for w in inhouse_tokens:
            #     inhouse_counts[w][k] += 1.0

            if not normal_tokens:
                continue

            vecs = self._embed_tokens(normal_tokens)
            class_vec_sums[k] += vecs.sum(axis=0)
            class_vec_counts[k] += vecs.shape[0]

        # クラスごとの平均方向 μ_k, 簡易 κ_k を推定
        self.mu = np.zeros((self.num_classes, D), dtype=np.float64)
        self.kappa = np.zeros(self.num_classes, dtype=np.float64)

        for k in range(self.num_classes):
            if class_vec_counts[k] == 0:
                continue
            mean_vec = class_vec_sums[k] / (class_vec_counts[k] + 1e-12)
            self.mu[k] = l2_normalize(mean_vec.reshape(1, -1))[0]
            R = np.linalg.norm(mean_vec)   # 平均ベクトル長 ≒ 平均コサイン類似度
            self.kappa[k] = 10.0 * R      # ざっくりしたスケーリング（チューニング可）

        # 社内語ごとの p(k|w) を推定
        # for w, counts in inhouse_counts.items():
        #     if counts.sum() == 0:
        #         continue
        #     probs = counts / counts.sum()
        #     self.inhouse_p_kw[w] = probs   # shape (K,)

    ########################################
    # 単語レベルの log p(x|k) （vMF 風）
    ########################################
    def _log_p_x_given_k(self, x):
        """
        x: shape (D,)
        戻り値: shape (K,) 各クラスの「vMF っぽい」対数スコア（正規化定数は無視）
        log p(x|k) ∝ κ_k * μ_k^T x
        """
        cos_sim = self.mu @ x  # (K,D) · (D,) -> (K,)
        return self.kappa * cos_sim

    ########################################
    # 文書のスコアリング
    ########################################
    def score_document(self, doc: str):
        tokens = self._tokenizer(doc)
        normal_tokens = tokens #= [t for t in tokens if t not in self.inhouse_terms]
        # inhouse_tokens = [t for t in tokens if t in self.inhouse_terms]

        log_likelihood = np.zeros(self.num_classes, dtype=np.float64)

        # 1. 埋め込みベース（通常トークンのみ）
        if normal_tokens:
            vecs = self._embed_tokens(normal_tokens)
            for x in vecs:
                x = l2_normalize(x.reshape(1, -1))[0]
                log_likelihood += self._log_p_x_given_k(x)

        # 2. 社内語由来のバイアス α * Σ log p(k|w)
        # if self.alpha_inhouse > 0 and inhouse_tokens:
        #     for w in inhouse_tokens:
        #         if w not in self.inhouse_p_kw:
        #             continue
        #         log_p_kw = np.log(self.inhouse_p_kw[w] + 1e-12)
        #         log_likelihood += self.alpha_inhouse * log_p_kw

        # 3. クラス事前確率 log p(k) を加える
        log_prior = np.log(self.class_priors + 1e-12)
        log_post_unnorm = log_prior + log_likelihood
        return log_post_unnorm

    def predict_proba(self, doc: str):
        log_scores = self.score_document(doc)
        log_probs = log_softmax(log_scores)
        return np.exp(log_probs)

    def predict(self, doc: str):
        probs = self.predict_proba(doc)
        k = int(np.argmax(probs))
        return self.categories[k]


#     # 7. 評価
#     print("Evaluating...")
#     y_pred = [clf.predict(doc) for doc in X_test]
#     acc = accuracy_score(y_test, y_pred)
#     print("Accuracy:", acc)
#     print(classification_report(y_test, y_pred, digits=3))

#     # 8. サンプル文で試す
#     sample = "ホワイトボードに在席状況を入力して、他部署からも確認できるようにしたい。"
#     print("sample:", sample)
#     print("tokens:", mecab_tokenizer(sample))
#     print("pred:", clf.predict(sample))
#     print("proba:", clf.predict_proba(sample))


In [35]:
import numpy as np
from collections import defaultdict
import pathlib
import MeCab
import unidic
from sentence_transformers import SentenceTransformer

############################################
# ユーティリティ
############################################

def l2_normalize(x: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(x, axis=-1, keepdims=True) + 1e-12
    return x / norm

def log_softmax(x):
    x = np.asarray(x, dtype=np.float64)
    x_max = np.max(x)
    log_sum_exp = np.log(np.sum(np.exp(x - x_max))) + x_max
    return x - log_sum_exp


############################################
# vMF 風のロバスト分類器（trimmed mean + λ(x) + 文書距離）
############################################

class RobustTextClassifier:
    """
    - 各カテゴリ k について、トークン埋め込みの代表方向 μ_k, 簡易集中度 κ_k を学習
      （vMF 分布の log p(x|k) ∝ κ_k * μ_k^T x を模している）
    - μ_k は「trimmed mean」（外れ方向を落とした平均）で推定
    - 推論時は、トークンごとに λ(x) を計算し、
      「どのクラスから見ても遠いベクトル」は log p への寄与を弱める
    - さらに、文書ベクトルの計算と文書間距離の計算機能を提供
    """

    # MeCab をクラス変数として 1 個だけ持つ
    dic_path = pathlib.Path(unidic.DICDIR)
    dic_dir = str(dic_path).replace("\\", "/")
    mecab = MeCab.Tagger(f'-r "{dic_dir}/mecabrc" -d "{dic_dir}"')
    mecab.parse("")  # GC 対策

    def __init__(
        self,
        categories,
        st_model: str = "stsb-xlm-r-multilingual",
        keep_pos: set[str] = {"名詞", "動詞", "形容詞", "副詞"},
        inhouse_terms=None,
        alpha_inhouse: float = 1.0,
        trim_ratio: float = 0.2,        # trimmed mean で落とす割合（低類似度側）
        lambda_low: float = 0.0,        # λ(x) が 0 になる cos 類似度の下限
        lambda_high: float = 0.8,       # λ(x) が 1 になる cos 類似度の上限
    ):
        """
        categories: カテゴリ名のリスト
        st_model: SentenceTransformer のモデル名
        keep_pos: 残す品詞の集合
        inhouse_terms: 社内語（固有名詞など）の set（いまは未使用でもOK）
        alpha_inhouse: 将来 p(k|w) を効かせるときの重み
        trim_ratio: 類似度の低い側から何割を trimmed mean で捨てるか
        lambda_low, lambda_high: λ(x) の線形スケール範囲
        """
        self.categories = list(categories)
        self.num_classes = len(categories)
        self.cat2idx = {c: i for i, c in enumerate(self.categories)}

        self.keep_pos = keep_pos
        self.st_model = SentenceTransformer(st_model)
        self.inhouse_terms = set(inhouse_terms or [])
        self.alpha_inhouse = alpha_inhouse

        self.trim_ratio = trim_ratio
        self.lambda_low = lambda_low
        self.lambda_high = lambda_high

        # パラメータ（fit でセット）
        self.mu = None              # (K, D) クラスごとの代表方向
        self.kappa = None           # (K,) 簡易集中度
        self.class_priors = None    # (K,) p(k)
        self.inhouse_p_kw = {}      # dict[w] -> np.array shape (K,)

    ########################################
    # tokenizer（MeCab + 品詞フィルタ + lemma）
    ########################################
    def _tokenizer(self, text: str):
        text = str(text).replace("\r", " ").replace("\n", " ")

        node = self.mecab.parseToNode(text)
        tokens = []

        while node:
            surface = node.surface
            # BOS/EOS や空文字はスキップ
            if not surface:
                node = node.next
                continue

            features = node.feature.split(",")
            pos = features[0]      # 品詞 大分類（名詞, 動詞, 助詞, など）

            # 名詞・動詞・形容詞・副詞 以外は捨てる
            if pos not in self.keep_pos:
                node = node.next
                continue

            # UniDic 系の典型的な並び:
            #  0: pos1, 1: pos2, 2: pos3, 3: pos4,
            #  4: cType, 5: cForm, 6: lForm, 7: lemma, 8: orth, 9: pron, ...
            lemma_idx = 7
            orth_idx  = 8

            token = surface  # デフォルトは表層形

            if len(features) > lemma_idx and features[lemma_idx] not in ("*", ""):
                token = features[lemma_idx]   # 漢字混じりの原形
            elif len(features) > orth_idx and features[orth_idx] not in ("*", ""):
                token = features[orth_idx]    # 文中表記

            tokens.append(token)
            node = node.next

        return tokens

    ########################################
    # 内部: トークン埋め込み
    ########################################
    def _embed_tokens(self, tokens):
        if not tokens:
            D = self.st_model.get_sentence_embedding_dimension()
            return np.zeros((0, D))
        vecs = self.st_model.encode(
            tokens,
            convert_to_numpy=True,
            normalize_embeddings=True,  # L2 正規化済み
        )
        return vecs

    ########################################
    # 内部: λ(x)（トークンごとの重み）を計算
    ########################################
    def _lambda_for_token(self, x: np.ndarray) -> float:
        """
        x: shape (D,), L2 正規化済み前提
        戻り値: 0.0〜1.0 の重み（どのクラスからも遠いトークンほど 0 に近づく）
        """
        if self.mu is None:
            return 1.0  # まだ学習前なら無条件に重み1

        cos_all = self.mu @ x            # shape: (K,)
        cos_max = float(np.max(cos_all)) # どのクラス方向とも似ていなければ小さい

        if self.lambda_high > self.lambda_low:
            lam = (cos_max - self.lambda_low) / (self.lambda_high - self.lambda_low)
            lam = float(np.clip(lam, 0.0, 1.0))
        else:
            lam = 1.0

        return lam

    ########################################
    # 学習（trimmed mean で μ_k を推定）
    ########################################
    def fit(self, docs, labels):
        """
        docs: 文書のリスト（文字列）
        labels: 各文書のカテゴリ名
        """
        label_idx = np.array([self.cat2idx[y] for y in labels])
        class_counts = np.bincount(label_idx, minlength=self.num_classes)
        self.class_priors = class_counts / class_counts.sum()

        D = self.st_model.get_sentence_embedding_dimension()

        # 各クラスごとのトークン埋め込みを全部ためる
        per_class_vecs = [[] for _ in range(self.num_classes)]

        for doc, y in zip(docs, labels):
            k = self.cat2idx[y]
            tokens = self._tokenizer(doc)
            normal_tokens = tokens

            if not normal_tokens:
                continue

            vecs = self._embed_tokens(normal_tokens)
            for v in vecs:
                per_class_vecs[k].append(v)

        # クラスごとの代表方向 μ_k と κ_k を trimmed mean で推定
        self.mu = np.zeros((self.num_classes, D), dtype=np.float64)
        self.kappa = np.zeros(self.num_classes, dtype=np.float64)

        for k in range(self.num_classes):
            if not per_class_vecs[k]:
                continue

            X = np.stack(per_class_vecs[k], axis=0)  # shape: (Nk, D)

            # 1) まず普通の平均で初期方向を作る
            mean0 = l2_normalize(X.mean(axis=0, keepdims=True))[0]

            # 2) 各ベクトルとの cos 類似度を計算
            cos = X @ mean0  # shape: (Nk,)

            # 3) 類似度の低い方から trim_ratio を落とす
            if 0.0 < self.trim_ratio < 0.5:
                thresh = np.quantile(cos, self.trim_ratio)
                mask = cos >= thresh
                X_trim = X[mask]
            else:
                X_trim = X

            # 4) trimmed mean を代表方向とする
            mean_vec = X_trim.mean(axis=0)
            self.mu[k] = l2_normalize(mean_vec.reshape(1, -1))[0]

            # 5) 平均ベクトル長 R から κ をざっくり決める
            R = np.linalg.norm(mean_vec)
            self.kappa[k] = 10.0 * R  # スケーリングはチューニングポイント

    ########################################
    # 単語レベルの log p(x|k) （vMF 風）
    ########################################
    def _log_p_x_given_k(self, x):
        """
        x: shape (D,)
        戻り値: shape (K,) 各クラスの「vMF っぽい」対数スコア（正規化定数は無視）
        log p(x|k) ∝ κ_k * μ_k^T x
        """
        cos_sim = self.mu @ x  # (K,D) · (D,) -> (K,)
        return self.kappa * cos_sim

    ########################################
    # 文書のスコアリング（λ(x) でノイズトークンを弱める）
    ########################################
    def score_document(self, doc: str):
        tokens = self._tokenizer(doc)
        normal_tokens = tokens

        log_likelihood = np.zeros(self.num_classes, dtype=np.float64)

        # 1. 埋め込みベース（トークンごと）
        if normal_tokens:
            vecs = self._embed_tokens(normal_tokens)
            for x in vecs:
                x = l2_normalize(x.reshape(1, -1))[0]
                lam = self._lambda_for_token(x)
                if lam <= 0.0:
                    continue  # 完全にノイズ扱い
                log_likelihood += lam * self._log_p_x_given_k(x)

        # 2. クラス事前確率 log p(k) を加える
        log_prior = np.log(self.class_priors + 1e-12)
        log_post_unnorm = log_prior + log_likelihood
        return log_post_unnorm

    def predict_proba(self, doc: str):
        log_scores = self.score_document(doc)
        log_probs = log_softmax(log_scores)
        return np.exp(log_probs)

    def predict(self, doc: str):
        probs = self.predict_proba(doc)
        k = int(np.argmax(probs))
        return self.categories[k]

    ########################################
    # 文書ベクトル（1つのベクトル）を計算
    ########################################
    def embed_document(self, doc: str, use_lambda_weight: bool = True) -> np.ndarray:
        """
        文書全体を 1 つのベクトルにまとめる。
        - トークン埋め込みの（重み付き）平均
        - use_lambda_weight=True のとき、
          λ(x) を重みとして「どのクラスからも遠いトークン」を自動的に弱める
        """
        tokens = self._tokenizer(doc)
        if not tokens:
            D = self.st_model.get_sentence_embedding_dimension()
            return np.zeros(D, dtype=np.float64)

        vecs = self._embed_tokens(tokens)  # shape: (N, D)
        if vecs.shape[0] == 0:
            D = self.st_model.get_sentence_embedding_dimension()
            return np.zeros(D, dtype=np.float64)

        # トークンごとの重み（デフォルト 1）
        weights = np.ones(vecs.shape[0], dtype=np.float64)

        if use_lambda_weight and self.mu is not None:
            for i in range(vecs.shape[0]):
                x = l2_normalize(vecs[i].reshape(1, -1))[0]
                weights[i] = self._lambda_for_token(x)

        # 全部 0 になってしまった場合は等重み平均にフォールバック
        if np.all(weights <= 0):
            weights = np.ones_like(weights)

        weights = weights / (weights.sum() + 1e-12)
        doc_vec = (weights[:, None] * vecs).sum(axis=0)
        doc_vec = l2_normalize(doc_vec.reshape(1, -1))[0]
        return doc_vec

    ########################################
    # 文書間の距離を計算
    ########################################
    def document_distance(
        self,
        doc1: str,
        doc2: str,
        metric: str = "cosine",
        use_lambda_weight: bool = True,
    ) -> float:
        """
        2つの文書 doc1, doc2 の距離を計算する。
        - まず embed_document で文書ベクトルを計算
        - metric:
          - "cosine": 1 - コサイン類似度
          - "euclidean": ユークリッド距離
        """
        v1 = self.embed_document(doc1, use_lambda_weight=use_lambda_weight)
        v2 = self.embed_document(doc2, use_lambda_weight=use_lambda_weight)

        if metric == "cosine":
            # embed_document は L2 正規化しているので dot のみでOK
            sim = float(np.dot(v1, v2))
            sim = max(min(sim, 1.0), -1.0)
            return 1.0 - sim
        elif metric == "euclidean":
            return float(np.linalg.norm(v1 - v2))
        else:
            raise ValueError(f"Unsupported metric: {metric}")


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# 1. データ読み込み
df = pd.read_csv("data/data.csv")
texts = df["text"].astype(str).tolist()
labels = df["category"].astype(str).tolist()

# 2. train / test に分割
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

rlf = RobustTextClassifier(
    categories=sorted(set(labels)),  # 社内語の p(k|w) をどれくらい効かせるか
)

rlf.fit(X_train, y_train)

print("Evaluating...")
y_pred = [rlf.predict(doc) for doc in X_test]
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)
print(classification_report(y_test, y_pred, digits=3))

# 8. サンプル文で試す
sample = "ホワイトボードに在席状況を入力して、他部署からも確認できるようにしたい。"
print("sample:", sample)
print("tokens:", mecab_tokenizer(sample))
print("pred:", rlf.predict(sample))
print("proba:", rlf.predict_proba(sample))



Evaluating...
Accuracy: 0.16279069767441862
              precision    recall  f1-score   support

         ESG      0.000     0.000     0.000        20
        サービス      0.000     0.000     0.000        22
        商品開発      0.000     0.000     0.000        13
          営業      0.000     0.000     0.000        26
        新規事業      0.000     0.000     0.000        14
          施工      0.237     0.250     0.243        36
          生産      0.000     0.000     0.000         5
     福利厚生・制度      0.000     0.000     0.000        59
          総務      0.151     0.868     0.258        38
          設計      0.000     0.000     0.000        25

    accuracy                          0.163       258
   macro avg      0.039     0.112     0.050       258
weighted avg      0.055     0.163     0.072       258

sample: ホワイトボードに在席状況を入力して、他部署からも確認できるようにしたい。
tokens: ['ホワイトボード-whiteboard', '在席', '状況', '入力', '為る', '他', '部署', '確認', '出来る', '為る']
pred: 総務


c:\Users\fujita096\PJ\TextClassificationSuite\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\fujita096\PJ\TextClassificationSuite\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\fujita096\PJ\TextClassificationSuite\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

proba: [0.02280771 0.01217608 0.00437027 0.10982883 0.00312859 0.19398225
 0.01735945 0.23393069 0.34333011 0.05908601]


In [33]:
res = rlf.predict_proba("ホワイトボードに在席状況を入力して、他部署からも確認できるようにしたい。")

y_pred = [rlf.predict_proba(doc) for doc in X_test]

count = 0
for X_test, y_pred in zip(X_test, y_pred):
    print(X_test)
    for cat, y in zip(rlf.categories, y_pred): 
        print(cat + ": " + str(y))
    print("---")

    count += 1
    if count > 20:
        break


地域に根差す産婆（助産院や自宅出産を扱う開業助産師を指す）は妊娠中から密に妊婦とその家族に寄り添う健診をして出産はもちろん産後やその後も相談役となり、産後うつや虐待の予防となる。 助産院は集いの場となり母親は子育ての仲間ができ安心した妊娠、出産から子育てと続き少子化対策や地域活性化につながる。 災害時の支援力も注目された。 そんな命を守り続ける存在の周知活動として無形文化遺産登録に向け活動中。
ESG: 0.0001224798842212269
サービス: 1.8183392354298897e-05
商品開発: 3.517567153490362e-07
営業: 0.040364919021515726
新規事業: 4.322642061452146e-07
施工: 0.32417260253473934
生産: 0.0040560613687497075
福利厚生・制度: 0.06681398772255664
総務: 0.5621810463378332
設計: 0.0022699357171027694
---
建売の計画をコンペ形式で実施したいです。
設計をされている方は学生時代に設計アイデアコンペに応募した方も多いのではないでしょうか。
全国から集まった選りすぐりのプランの建売コンペはやりがい、見ごたえ共に十分かと思います。
実例を評価するコンペはありますが、アイデア・構想の設計コンペはあまりないのではと思います。
また、本アイデアが入賞した際の賞金を建売コンペ1位の方への賞金としたいと考えています。
ESG: 0.0003191921001668419
サービス: 3.30493555084513e-05
商品開発: 3.1883868283791865e-06
営業: 0.03931154975963855
新規事業: 1.3202049949196588e-06
施工: 0.3167025584081201
生産: 0.010958343425647844
福利厚生・制度: 0.07345535278677465
総務: 0.5353667366985726
設計: 0.023848708873742217
---
クリエイティブな業務を行う設計やコーディネーターは、良いものを作るためにはやる気・モチベーションが大切です。モチベーションアップのた